# Calculating Betweenness

This notebook will walk you through the process of calculating betweenness values with the Madina
package. It will read in geographic data as well as global parameters from a local directory, load
the layers into a Zonal object, generate the geographic network, and run betweenness analysis
on the given origin and destination layers, potentially with elastic weights.

Please use this notebook as a template to create your own workflow, and feel free to change the
behavior of the logger, or add any necessary preprocessings to the geographic layers.

To run this notebook on your own data, please put it in a folder parallel to the `madina` subfolder
under the repository as shown below. If you want to run the notebook elsewhere, remember to change
the `sys.path.append("..")` to append the parent folder of the `madina` subfolder to system path.
```
madina (the repository)
|---madina
|   |---all code for the package
|---notebooks
    |---this notebook
```

## Importing necessary packages

The import is split into two sections: importing dependencies, and importing the Madina package.
We recommend importing classes and functions from the Madina package directly instead of importing
the Madina package as a whole. We also recommend that the functions are imported from local files
for now, in order to avoid the troubles coming with installation and conflicts between conda and
pip.

In [2]:
import sys
import pandas as pd
import geopandas as gpd
from datetime import datetime
from pathlib import Path
from shapely.ops import transform

sys.path.append("../../..")
from madina.zonal.zonal import Layer
from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight

## Creating a Logger

The Logger class provides a simple interface to show the users progress of the betweenness analysis,
as well as to debug according to the intermediate results it saved.

In [3]:
class Logger():
    def __init__(self, output_folder):
        self.output_folder = output_folder
        self.start_time = datetime.now()
        self.log_df = pd.DataFrame(
            {
                "time": pd.Series(dtype='datetime64[ns]'),
                "distance": pd.Series(dtype="string"),
                "tune_penalty": pd.Series(dtype="string"),
                "elastic_weight": pd.Series(dtype="string"),
                "origin": pd.Series(dtype="string"),
                "destination": pd.Series(dtype="string"),
                "event": pd.Series(dtype="string")
            }
        )

    def simulation_start(self, zone, network_weight_settings, turn_penalty_settings, elastic_weight_settings):
        self.betweenness_record = zone.layers['streets'].gdf.copy(deep=True)
        self.separate_simulation_records = {}
        for network_weight in network_weight_settings:
            self.separate_simulation_records[network_weight] = {}
            for turn_penalty in turn_penalty_settings:
                self.separate_simulation_records[network_weight][turn_penalty] = {}
                for elastic_weight in elastic_weight_settings:
                    self.separate_simulation_records[network_weight][turn_penalty][elastic_weight] = self.betweenness_record.copy(deep=True)

    def log(self, input_dict):
        input_dict["time"] = datetime.now()
        if self.log_df.shape[0] == 0:
            print(f"total time\tseconds elapsed\tdiatance method\telastic_weight\t{'origin':^15s}\t{'destination':^15s}\tevent")
            input_dict["seconds_elapsed"] = 0
            input_dict["cumulative_seconds"] = 0
        else:
            time_elapsed = (input_dict["time"] - self.log_df.iloc[-1]["time"]).seconds
            input_dict["seconds_elapsed"] = time_elapsed
            input_dict["cumulative_seconds"] = self.log_df["seconds_elapsed"].sum() + time_elapsed

        for column_name in self.log_df.columns:
            if column_name not in input_dict:
                input_dict[column_name] = "---"

        self.log_df = pd.concat([self.log_df, pd.DataFrame([input_dict])], ignore_index=True)
        print(
            f"{input_dict['cumulative_seconds']:6.4f}\t\t"
            f"{input_dict['seconds_elapsed']}\t\t\t\t"
            f"{input_dict['distance']}\t\t"
            f"{input_dict['tune_penalty']}\t\t"
            f"{input_dict['elastic_weight']}\t\t"
            f"{input_dict['origin']:^15s}\t"
            f"{input_dict['destination']:^15s}\t"
            f"{input_dict['event']}"
        )

    def pairing_end(self, shaqra: Zonal, pairing, network_weight, turn_penalty, elastic_weight):
        """
        This function should be called whenever one round of betweenness analysis concludes.
        It will save the betweenness results for the current round into a subfolder of the output
        folder designated by the Zonal class, as well as logging the execution times.
        """
        # creating a folder for output
        pairing_folder = self.output_folder + f"{network_weight}\\{'with_turns' if turn_penalty else 'no_turns'}\\{'elastic_weight' if elastic_weight else 'unadjusted_weight'}\\O({pairing['Origin_Name']})_D({pairing['Destination_Name']})\\"
        Path(pairing_folder).mkdir(parents=True, exist_ok=True)

        street_gdf = shaqra.layers["streets"].gdf
        node_gdf = shaqra.network.nodes
        origin_gdf = node_gdf[node_gdf["type"] == "origin"]
        destination_gdf = node_gdf[node_gdf["type"] == "destination"]
        edge_gdf = shaqra.network.edges
        edge_gdf["width"] = edge_gdf["betweenness"] / edge_gdf["betweenness"].mean() + 0.25


        self.betweenness_record = self.betweenness_record.join(
            edge_gdf[['parent_street_id', 'betweenness']].set_index('parent_street_id')).rename(
            columns={
                "betweenness": f"{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'}_{pairing['Between_Name']}"})

        self.separate_simulation_records[network_weight][turn_penalty][elastic_weight] = \
        self.separate_simulation_records[network_weight][turn_penalty][elastic_weight].join(
            edge_gdf[['parent_street_id', 'betweenness']].set_index('parent_street_id')).rename(
            columns={
                "betweenness": f"{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'}_{pairing['Between_Name']}"})

        save_results = \
            edge_gdf.set_index('parent_street_id').join(street_gdf, lsuffix='_from_edge')[
                # , rsuffix='_from_streets'
                ["betweenness", "__GUID", "geometry"]]
        save_results = save_results.rename(
            columns={
                "betweenness": f"{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'}_{pairing['Between_Name']}"})

        save_results.to_csv(pairing_folder + "flows.csv")
        # save_results.to_file(pairing_folder + "flows.geojson", driver="GeoJSON")
        self.betweenness_record.to_csv(pairing_folder + "betweenness_record_so_far.csv")
        self.separate_simulation_records[network_weight][turn_penalty][elastic_weight].to_csv(
            pairing_folder + "simulation_record_so_far.csv")

        self.log_df.to_csv(pairing_folder + "time_log.csv")

        self.log({
            "origin": pairing["Origin_Name"],
            "destination": pairing["Destination_Name"],
            "event": "Output saved",
            "distance": network_weight,
            "tune_penalty": turn_penalty,
            "elastic_weight": elastic_weight})

    def simulation_end(self, network_weight_settings, turn_penalty_settings, elastic_weight_settings):
        self.betweenness_record.to_csv(self.output_folder + "betweenness_record.csv")
        #self.betweenness_record.to_file(self.output_folder + "street_network_betweenness_record.geojson", driver="GeoJSON")
        for network_weight in network_weight_settings:
            for turn_penalty in turn_penalty_settings:
                for elastic_weight in elastic_weight_settings:
                    pairing_folder = self.output_folder + f"{network_weight}\\{'with_turns' if turn_penalty else 'no_turns'}\\{'elastic_weight' if elastic_weight else 'unadjusted_weight'}\\"
                    self.separate_simulation_records[network_weight][turn_penalty][elastic_weight].to_csv(pairing_folder+"betweenness_record.csv")
        self.log({"event": "All DONE."})
        self.log_df.to_csv(self.output_folder + "time_log.csv")


## Defining the workflow

The function below defines the entire workflow of betweenness analysis, from data loading to
saving the calculated flows. See comments for details.

In [4]:
def betweenness_flow_simulation(
        city_name="Somerville",
        data_folder=None,
        output_folder=None,
        pairings_file="Pairings.csv",
        network_weight_settings = ["Perceived", "Geometric"],
        turn_penalty_settings = [False, True],
        elastic_weight_settings = [False, True],
        num_cores=8,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs=None
    ):
    # Find the data folder and output folder according to user inputs.
    if data_folder is None:
        data_folder = "Cities\\"+city_name+"\\Data\\"
    if output_folder is None:
        start_time = datetime.now()
        output_folder = f"Cities\\{city_name}\\Simulations\\{start_time.year}_{start_time.month:02d}_{start_time.day:02d}_{start_time.hour:02d}_{start_time.minute:02d}\\"

    # Initializing the Logger
    logger=Logger(output_folder)
    logger.log({"event": "beginning"})

    # Reading the pairings file, which defines all origin-destination pairs whose betweenness is
    # to be analyzed
    pairings = gpd.read_file(data_folder + pairings_file)

    # Initializing the Zonal object
    zonal = Zonal()
    if impose_crs is not None:
        zonal = Zonal(projected_crs=impose_crs)

    # Loading the network layer from the file designated in the pairings file
    zonal.load_layer(
        layer_name='streets',
        file_path=data_folder +  pairings.at[0, "Network_File"]
    )
    if impose_crs is not None:
        zonal.layers["streets"] = Layer(
            label="streets",
            gdf=gpd.read_file(data_folder +  pairings.at[0, "Network_File"]).set_crs(impose_crs, allow_override=True),
            show=True,
            original_crs=impose_crs,
            file_path=(data_folder + pairings.at[0, "Network_File"])
        )

    # Cleaning the network layer
    geometry_gdf = zonal.layers["streets"].gdf
    polygon_idxs = geometry_gdf[geometry_gdf["geometry"].geom_type == "Polygon"].index
    geometry_gdf.loc[polygon_idxs,"geometry"] = geometry_gdf.loc[polygon_idxs, "geometry"].exterior
    zonal.layers["streets"].gdf = geometry_gdf

    if zonal.layers["streets"].gdf.has_z.any():
        def _to_2d(x, y, z):
            return tuple(filter(None, [x, y]))
        zonal.layers["streets"].gdf["geometry"] = zonal.layers["streets"].gdf[
            "geometry"].apply(lambda s: transform(_to_2d, s))

    if (zonal.layers["streets"].gdf.geometry.geom_type == "MultiLineString").all():
        zonal.layers["streets"].gdf["geometry"] = zonal.layers["streets"].gdf[
            "geometry"].apply(lambda s: s.geoms[0])

    print(f'streets\t{zonal.layers["streets"].gdf.crs}')

    logger.simulation_start(zonal, network_weight_settings, turn_penalty_settings, elastic_weight_settings)

    # Setting up a street network
    zonal.create_street_network(
        source_layer="streets",
        node_snapping_tolerance=1,
        weight_attribute=None if "Perceived" not in network_weight_settings else pairings.at[0, "Network_Cost"],
        discard_redundant_edges=True,
        turn_threshold_degree=turn_threshold_degree,
        turn_penalty_amount=turn_penalty_amount
        )
    # This is to re-set the origins and destinations before any new iteration.
    clean_node_gdf = zonal.network.nodes.copy(deep=True)

    # preparing percieved and geometric weights...
    perceived_network_weight = zonal.network.edges["weight"]
    perceived_network_weight = perceived_network_weight.apply(lambda x: max(1, x))
    geometric_network_weight = zonal.network.edges["geometry"].length

    logger.log({"event": "Network topology created."})

    for idx, pairing in pairings.iterrows():
        # Begin the execution of one O-D pair designated in the pairings file
        if pairing["Origin_Name"] not in zonal.layers:
            # Loading the origin layer if it is not already loaded.
            zonal.load_layer(
                layer_name=pairing["Origin_Name"],
                file_path=data_folder + pairing["Origin_File"]
            )
            print(f"{pairing['Origin_Name']}\t{zonal.layers[pairing['Origin_Name']].gdf.crs}")
            if (impose_crs is not None) and (zonal.layers[pairing["Origin_Name"]].gdf.crs != impose_crs):
                print("Imposing CRS", impose_crs)
                zonal.layers[pairing["Origin_Name"]] = Layer(
                    label=pairing["Origin_Name"],
                    gdf=gpd.read_file(data_folder + pairing["Origin_File"]).set_crs(impose_crs, allow_override=True),
                    show=True,
                    original_crs=impose_crs,
                    file_path=(data_folder + pairing["Origin_File"])
                )

            print(f"{pairing['Origin_Name']}\t{zonal.layers[pairing['Origin_Name']].gdf.crs}\t{zonal.layers[pairing['Origin_Name']].crs}")


            # Data Cleaning
            if zonal.layers[pairing["Origin_Name"]].gdf.has_z.any():
                zonal.layers[pairing["Origin_Name"]] = Layer(
                    label=pairing["Origin_Name"],
                    gdf=zonal.layers[pairing["Origin_File"]].gdf["geometry"].apply(lambda s: transform(_to_2d, s)),
                    show=True,
                    original_crs=impose_crs,
                    file_path=(data_folder + pairing["Origin_File"])
                )

        if pairing["Destination_Name"] not in zonal.layers:
            # Loading the destination layer if it is not already loaded.
            zonal.load_layer(
                layer_name=pairing["Destination_Name"],
                file_path=data_folder + pairing["Destination_File"]
            )
            if (impose_crs is not None) and (zonal.layers[pairing["Destination_Name"]].gdf.crs != impose_crs):
                zonal.layers[pairing["Destination_Name"]] = Layer(
                    label=pairing["Destination_Name"],
                    gdf=gpd.read_file(data_folder + pairing["Destination_File"]).set_crs(impose_crs, allow_override=True),
                    show=True,
                    original_crs=impose_crs,
                    file_path=(data_folder + pairing["Destination_File"])
                )
            # Data Cleaning
            if zonal.layers[pairing["Destination_Name"]].gdf.has_z.any():
                zonal.layers[pairing["Destination_Name"]] = Layer(
                    label=pairing["Destination_Name"],
                    gdf=zonal.layers[pairing["Destination_File"]].gdf["geometry"].apply(lambda s: transform(_to_2d, s)),
                    show=True,
                    original_crs=impose_crs,
                    file_path=(data_folder + pairing["Destination_File"])
                )
        
        print(f"{pairing['Origin_Name']}\t{zonal.layers[pairing['Origin_Name']].gdf.crs}")
        print(f"{pairing['Destination_Name']}\t{zonal.layers[pairing['Destination_Name']].gdf.crs}")
        print(f"Zonal Edge Count: {len(zonal.network.edges)}")

        # making sure to clear any existing origins and destinations before adding new ones.
        zonal.network.nodes = clean_node_gdf.copy(deep=True)

        # iterating over the three betweenness calculation options: network weight, turn penalty,
        # and elastic weight
        for network_weight in network_weight_settings:

            # setting the proper network_weight
            if network_weight == "Perceived":
                zonal.network.nodes["weight"] = perceived_network_weight
            elif network_weight == "Geometric":
                zonal.network.edges["weight"] = geometric_network_weight

            # Insert the origin and destination nodes into the network
            zonal.insert_node(
                label="origin",
                layer_name=pairing["Origin_Name"],
                weight_attribute=pairing["Origin_Weight"] if pairing["Origin_Weight"] != "Count" else None,
            )

            zonal.insert_node(
                label="destination",
                layer_name=pairing["Destination_Name"],
                weight_attribute=pairing["Destination_Weight"] if pairing["Destination_Weight"] != "Count" else None,
            )

            inelastic_weight = zonal.network.nodes['weight']


            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Origins and Destinations prepared."
                })

            zonal.create_graph(light_graph=True, od_graph=True)

            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Light and dense graphs prepared."
                })

            for turn_penalty in turn_penalty_settings:
                for elastic_weight in elastic_weight_settings:
                    # The order of these is important, as the weight is overriden by
                    # elastic weight as there is no clean way to update weight for now.
                    if elastic_weight:
                        # Calculate the elastic weight
                        get_elastic_weight(
                            zonal.network,
                            search_radius=800,
                            detour_ratio=0.002,
                            beta=0.002,
                            decay=True,
                            turn_penalty=turn_penalty,
                            retained_d_idxs=None
                            )

                        logger.log({
                            "origin": pairing["Origin_Name"],
                            "destination": pairing["Destination_Name"],
                            "event": "Elastic Weights generated.",
                            "distance": network_weight, "tune_penalty": turn_penalty,
                            "elastic_weight": elastic_weight})
                    else:
                        zonal.network.nodes['weight'] = inelastic_weight

                    # Run the betwenness analysis

                    node_gdf = zonal.network.nodes
                    origin_gdf = node_gdf[node_gdf["type"] == "origin"]

                    num_cores = min(origin_gdf.shape[0], num_cores)
                    betweenness_output = parallel_betweenness(
                        zonal.network,
                        search_radius=float(pairing["Radius"]),
                        detour_ratio=float(pairing["Detour"]),
                        decay=False if elastic_weight else True,
                        decay_method="exponent",  # "power", "exponent"
                        beta=float(pairing["Beta"]),
                        path_detour_penalty="equal",  # "power", "exponent", "equal"
                        origin_weights=True,
                        closest_destination=False,
                        destination_weights=True,
                        num_cores=num_cores,
                        turn_penalty=turn_penalty,
                        rertain_expensive_data=False if elastic_weight else True,
                        retained_d_idxs=None,
                        retained_paths=None,
                        retained_distances=None,
                    )

                    if not elastic_weight:
                        retained_d_idxs = betweenness_output["retained_d_idxs"]

                    logger.log({
                        "origin": pairing["Origin_Name"],
                        "destination": pairing["Destination_Name"],
                        "event": "Betweenness estimated.",
                        "distance": network_weight,
                        "tune_penalty": turn_penalty,
                        "elastic_weight": elastic_weight})
                    logger.pairing_end(zonal, pairing, network_weight, turn_penalty, elastic_weight)
                    
    logger.simulation_end(network_weight_settings, turn_penalty_settings, elastic_weight_settings)

## Running the workflow

The cell below shows one sample of running the betweenness analysis on the city of Somerville in
Massachusetts, USA. The data folder used can be downloaded from here: TODO: Dropbox link

In [5]:
betweenness_flow_simulation(
    city_name="Somerville",
    data_folder=f'..\\..\\..\\test_ipynb\\Cities\\Somerville\\Data\\',
    pairings_file="Pairings.csv",
    network_weight_settings=["Geometric"],          # ["Perceived", "Geometric"],
    turn_penalty_settings=[True],                   # [False, True]
    elastic_weight_settings=[False],          # [False, True]
    num_cores=20,
    turn_threshold_degree=45,
    turn_penalty_amount=62.3,
    impose_crs='epsg:26986'
)

total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	epsg:26986


c:\Users\56830\OneDrive - Massachusetts Institute of Technology\UROPs\UNA\madina\docs\source\tutorial\../../..\madina\zonal\network_utils.py:171: FutureWarning: The `query_bulk()` method is deprecated and will be removed in GeoPandas 1.0. You can use the `query()` method instead.
  matching = point_geometries.sindex.query_bulk(


21.0000		21				---		---		---		      ---      	      ---      	Network topology created.
Bus	EPSG:26986
Bus	EPSG:26986	EPSG:26986
Bus	EPSG:26986
Subway	EPSG:26986
Zonal Edge Count: 10122
21.0000		0				---		---		---		      Bus      	    Subway     	Origins and Destinations prepared.
22.0000		1				---		---		---		      Bus      	    Subway     	Light and dense graphs prepared.
-------------------------------------------
All cores done in 7.65
-------------------------------------------
29.0000		7				Geometric		True		False		      Bus      	    Subway     	Betweenness estimated.
29.0000		0				Geometric		True		False		      Bus      	    Subway     	Output saved
Home	EPSG:26986
Home	EPSG:26986	EPSG:26986
Home	EPSG:26986
Subway	EPSG:26986
Zonal Edge Count: 10122
31.0000		2				---		---		---		     Home      	    Subway     	Origins and Destinations prepared.
47.0000		16				---		---		---		     Home      	    Subway     	Light and dense graphs prepared.
-------------------------------------------
A